In [1]:
# top level imports
from pathlib import Path
import os, sys
import itertools

import parse

import napari

from skimage.measure import (regionprops, _regionprops)

from napari.utils.notebook_display import nbscreenshot

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats import (_my_props_to_dict, _assert_uint16_labels)
from infer_subc.utils.stats_helpers import *

from infer_subc.organelles import * 

from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH ) 

import time
%load_ext autoreload
%autoreload 2

## Get and load image for quantification
Specifically, this will include the raw image and the outputs from segmentation.

In [2]:
test_img_n = TEST_IMG_N

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

raw_data_path = data_root_path / "raw"
im_type = ".czi"

raw_file_list = list_image_files(raw_data_path,im_type)
raw_img_name = raw_file_list[test_img_n]

# adding an additional list of image paths for the matching segmentation files
seg_data_path = data_root_path / "out"
seg_file_list = list_image_files(seg_data_path, "tiff")

# changing output directory for this notebook to a new folder called "quant"
out_data_path = data_root_path / "quant"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [3]:
# raw image
raw_img_data, raw_meta_dict = read_czi_image(raw_img_name)

channel_names = raw_meta_dict['name']
img = raw_meta_dict['metadata']['aicsimage']
scale = raw_meta_dict['scale']
channel_axis = raw_meta_dict['channel_axis']

In [4]:
## For each import, change the string to match the suffix on the segmentation files (i.e., the stuff following the "-")

# masks
nuc_seg = import_inferred_organelle("20230426_test_nuc", raw_meta_dict, seg_data_path)
cell_seg = import_inferred_organelle("20230426_test_cell", raw_meta_dict, seg_data_path)
cyto_seg = import_inferred_organelle("20230426_test_cyto", raw_meta_dict, seg_data_path)
# mask_seg = import_inferred_organelle("masks", raw_meta_dict, seg_data_path)

#organelles
lyso_seg = import_inferred_organelle("20230426_test_lyso", raw_meta_dict, seg_data_path)
mito_seg = import_inferred_organelle("20230426_test_mito", raw_meta_dict, seg_data_path)
golgi_seg = import_inferred_organelle("20230426_test_golgi", raw_meta_dict, seg_data_path)
perox_seg = import_inferred_organelle("20230426_test_perox", raw_meta_dict, seg_data_path)
ER_seg = import_inferred_organelle("20230426_test_ER", raw_meta_dict, seg_data_path)
LD_seg = import_inferred_organelle("20230426_test_LD", raw_meta_dict, seg_data_path)


loaded  inferred 3D `20230426_test_nuc`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cell`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_cyto`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_lyso`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_mito`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_golgi`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_perox`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_ER`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 
loaded  inferred 3D `20230426_test_LD`  from C:\Users\Shannon\Documents\Python_Scripts\Infer-subc\out 


-------------------------
## Visualize the image and segmentation outputs

In [5]:
viewer = napari.Viewer()

22-Sep-23 08:10:58 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 1086x657+634+358 (frame: 1104x704+625+320) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY1". Resulting geometry: 905x825+632+351 (frame: 923x872+623+313) margins: 9, 38, 9, 9 minimum size: 385x515 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=403,562 maxtrack=0,0)


In [6]:
viewer.add_image(raw_img_data)

<Image layer 'raw_img_data' at 0x1f648a95060>

In [7]:
viewer.add_image(cell_seg, colormap='gray', opacity=0.3)

viewer.add_image(nuc_seg, colormap='blue', opacity=0.3)
viewer.add_image(lyso_seg, colormap='cyan', opacity=0.3)
viewer.add_image(mito_seg, colormap='green', opacity=0.3)
viewer.add_image(golgi_seg, colormap='yellow', opacity=0.3)
viewer.add_image(perox_seg, colormap='bop orange', opacity=0.3)
viewer.add_image(ER_seg, colormap='red', opacity=0.3)
viewer.add_image(LD_seg, colormap='magenta', opacity=0.3)

<Image layer 'LD_seg' at 0x1f68303f6d0>

In [8]:
viewer.close()

-------------------------
## Exploring the regionprops measurements

### Some regionprops measurements are not possible in 3D or with real world units:

> Below, we have listed the properties that are supported in 3D and the properties that support scaling to real-world units.
>
> NOTE: 
> The regionprop property names correspond to 2D measurements even for those that are able to function in 3D (i.e. "area" is actually "volume" when a 3D image is being processed, etc.).

In [9]:
labels = label(nuc_seg )
rp = regionprops(labels, intensity_image=raw_img_data[NUC_CH], spacing=scale)

supported = [] 
unsupported = []

for prop in rp[0]:
    try:
        rp[0][prop]
        supported.append(prop)
    except NotImplementedError:
        unsupported.append(prop)

print("Supported properties for 3D analysis:")
print("  " + "\n  ".join(supported))
print()
print("Properties only supported in 2D:")
print("  " + "\n  ".join(unsupported))

Supported properties for 3D analysis:
  area
  area_bbox
  area_convex
  area_filled
  axis_major_length
  axis_minor_length
  bbox
  centroid
  centroid_local
  centroid_weighted
  centroid_weighted_local
  coords
  equivalent_diameter_area
  euler_number
  extent
  feret_diameter_max
  image
  image_convex
  image_filled
  image_intensity
  inertia_tensor
  inertia_tensor_eigvals
  intensity_max
  intensity_mean
  intensity_min
  label
  moments
  moments_central
  moments_normalized
  moments_weighted
  moments_weighted_central
  moments_weighted_normalized
  slice
  solidity

Properties only supported in 2D:
  eccentricity
  moments_hu
  moments_weighted_hu
  orientation
  perimeter
  perimeter_crofton


In [ ]:
## To-do: need to include Rene's list of scaled vs. unscalled measurements!!

-------------------------
## **ORGANELLE METRICS**

To measure the amount, size, and shape of the individual organelles, we will utilize `skimage.measure.regionprops`. These measurements can be collected based on pixel/voxel units (assuming the image is isotropic in all dimensions) and or "real-world" units. Since most confocal microscope images are anisotropic (mostly with respect to the Z dimension), we will preferentially utilize real-world units. Luckily, regionprops>=0.20.0 has incorporated a spacing parameter that can handle anisotropic data.

Similar concepts will also be applied to measure the cell, cytoplasm, and nucleus below as well as organelle contacts in notebook {???}.

### 0. Apply cell mask
To ensure we are performing single cell analysis, we will apply the cell segmentation as a mask.

In [ ]:
nuc_masked = apply_mask(nuc_seg, cell_seg)

### 1. Build the list of measurements we want to include from regionprops

In [ ]:
# start with LABEL
test_properties = ["label"]

# add position
test_properties = test_properties + ["centroid", "bbox"]

# add area
test_properties = test_properties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements
test_properties = test_properties + ["extent", "euler_number", "convex_area", "solidity", "axis_major_length"] # "feret_diameter_max", "axis_minor_length"]

# add intensity values (used for quality checks only)
test_properties = test_properties + ["min_intensity", "max_intensity", "mean_intensity"]

test_properties

['label',
 'centroid',
 'bbox',
 'area',
 'equivalent_diameter',
 'extent',
 'euler_number',
 'convex_area',
 'solidity',
 'axis_major_length',
 'min_intensity',
 'max_intensity',
 'mean_intensity']

### 2. Add additional measurements as "extra_properties" with custom functions.

In [ ]:
# create a function to include the standard deviation of intensities (for quality checks only)
def _standard_deviation_intensity(region, intensities):
    return np.std(intensities[region])

test_extra_properties = [_standard_deviation_intensity]

In [ ]:
# creating a function to slightly alter the existing minor_axis_length measurement



### 3. Run regionprops and export values as a pandas dataframe

In [ ]:
test_rp = regionprops(label_image=nuc_masked,
                      intensity_image=raw_img_data[0], 
                      extra_properties=test_extra_properties, 
                      spacing=scale)

In [ ]:
test_props = _my_props_to_dict(test_rp, 
                               label_image=nuc_masked, 
                               intensity_image=raw_img_data[0], 
                               properties=test_properties, 
                               extra_properties=test_extra_properties,
                               spacing=scale)

`TO DO: figure out if regionprops_table() function from skimage.measure._regionprops can just be used instead of the combination of the two function above.`

In [ ]:
test_props_table = pd.DataFrame(test_props)

test_props_table.rename(columns={"area": "volume"}, inplace=True)

round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
test_props_table.insert(loc=1, column="scale", value=f"{round_scale}")

### 4. Add additional measurements

In [ ]:
# creating a function to measure the surface area of each object. This function utilizes "marching_cubes" to generate a mesh (non-pixelated object)
def _surface_area_from_props(labels, props):
    # SurfaceArea
    surface_areas = np.zeros(len(props["label"]))
    # TODO: spacing = [1, 1, 1] # this is where we could deal with anisotropy in Z

    for index, lab in enumerate(props["label"]):
        # this seems less elegant than you might wish, given that regionprops returns a slice,
        # but we need to expand the slice out by one voxel in each direction, or surface area freaks out
        volume = labels[
            max(props["bbox-0"][index] - 1, 0) : min(props["bbox-3"][index] + 1, labels.shape[0]),
            max(props["bbox-1"][index] - 1, 0) : min(props["bbox-4"][index] + 1, labels.shape[1]),
            max(props["bbox-2"][index] - 1, 0) : min(props["bbox-5"][index] + 1, labels.shape[2]),
        ]
        volume = volume == lab
        verts, faces, _normals, _values = marching_cubes(
            volume,
            method="lewiner",
            spacing=(1.0,) * labels.ndim,
            level=0,
        )
        surface_areas[index] = mesh_surface_area(verts, faces)

    return surface_areas

In [ ]:
# creating a function to measure the volume of the surface area mesh which is slightly different than the volume collected in region props; this will be used for a more accurate surface area to volume ratio.



In [ ]:
# props["surface_area"] = surface_area_from_props(nuc_seg, props)
test_surface_area_tab = pd.DataFrame(_surface_area_from_props(nuc_masked, test_props))

test_props_table.insert(11, "surface_area", test_surface_area_tab)

In [ ]:
pd.set_option('display.max_columns', None)

test_props_table

,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,_standard_deviation_intensity
0,5,"(0.3891, 0.0799, 0.0799)",2.628936,31.602057,28.150081,0,284,242,16,504,463,64248.140625,1037.508176,12.560231,0.537266,1,1135.780394,0.913476,21.135161,0.0,8392.0,2012.255031,1048.054565


### Define `get_org_metrics_3D()` function

Based on the _prototyping_ above define the function to quantify amount, size, and shape of organelles.

In [ ]:
def _get_org_metrics_3D(segmentation_img: np.ndarray, intensity_img, mask: np.ndarray, scale: Union[tuple, None]=None) -> Tuple[Any, Any]:
    """
    Parameters
    ------------
    segmentation_img:
        a 3d np.ndarray image of the segemented organelles
    intensity_img:
        a 3d np.ndarray image of the "raw" florescence intensity the segmentation was based on
    mask:
        a 3d np.ndarray image of the cell mask (or other mask of choice); used to create a "single cell" analysis

    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each object in the segmentation image (rows) and the regionprops object

    Regionprops measurements included:
    ['label',
    'centroid',
    'bbox',
    'area',
    'equivalent_diameter',
    'extent',
    'feret_diameter_max',
    'euler_number',
    'convex_area',
    'solidity',
    'axis_major_length',
    'axis_minor_length',
    'max_intensity',
    'mean_intensity',
    'min_intensity']

    Additional measurement include:
    ['standard_deviation_intensity',
    'surface_area']
    """
    ###################################################
    ## MASK THE ORGANELLE OBJECTS THAT WILL BE MEASURED
    ###################################################
    # in case we sent a boolean mask (e.g. cyto, nucleus, cellmask)
    input_labels = _assert_uint16_labels(segmentation_img)

    # mask
    input_labels = apply_mask(segmentation_img, mask)

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]

    # add position
    properties = properties + ["centroid", "bbox"]

    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

    # add shape measurements
    properties = properties + ["extent", "euler_number", "convex_area", "solidity", "axis_major_length"] # ,"feret_diameter_max", "axis_minor_length"]

    # add intensity values (used for quality checks)
    properties = properties + ["min_intensity", "max_intensity", "mean_intensity"]

    #######################
    ## ADD EXTRA PROPERTIES
    #######################
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])

    extra_properties = [standard_deviation_intensity]

    ##################
    ## RUN REGIONPROPS
    ##################
    rp = regionprops(input_labels, 
                     intensity_image=intensity_img, 
                     extra_properties=extra_properties, 
                     spacing=scale)

    props = _my_props_to_dict(rp, 
                              label_image=input_labels, 
                              intensity_image=intensity_img, 
                              properties=properties, 
                              extra_properties=extra_properties,
                              spacing=scale)

    props_table = pd.DataFrame(props)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(loc=1, column="scale", value=f"{round_scale}")

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(surface_area_from_props(input_labels, props))

    props_table.insert(11, "surface_area", surface_area_tab)

    ################################################################
    ## ADD SKELETONIZATION OPTION FOR MEASURING LENGTH AND BRANCHING
    ################################################################
    #  # ETC.  skeletonize via cellprofiler /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/morphologicalskeleton.py
    #         if x.volumetric:
    #             y_data = skimage.morphology.skeletonize_3d(x_data)
    # /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/measureobjectskeleton.py

    return props_table, rp

In [ ]:
org_img = raw_img_data[0]
org_obj = nuc_seg
cell_mask = cell_seg

nuc_table, rp = _get_org_metrics_3D(segmentation_img=org_obj, intensity_img=org_img, mask=cell_mask, scale=scale)

In [ ]:
# Will result in FALSE because the _standard_deviation_intensity function was used in the prototyping 
# instead of the standard_deviation_intensity function from stats.py
nuc_table.equals(test_props_table)

False

### Add prototype function into `stats.py`

In [ ]:
from infer_subc.utils.stats import get_org_metrics_3D

nuc_table_final, rp = get_org_metrics_3D(org_obj, org_img, cell_mask, scale=scale)

In [ ]:
nuc_table.equals(nuc_table_final)

True

In [ ]:
nuc_table

,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,surface_area,volume,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity,max_intensity,mean_intensity,standard_deviation_intensity
0,5,"(0.3891, 0.0799, 0.0799)",2.628936,31.602057,28.150081,0,284,242,16,504,463,64248.140625,1037.508176,12.560231,0.537266,1,1135.780394,0.913476,21.135161,0.0,8392.0,2012.255031,1048.054565


-------------------------
## **CELL REGIONS METRICS**
Here we are creating a function to measure the cell, cytoplasm, and nucleus. These measures will be beneficial in seeing changes to the cell/nucleus shape, but also to normalize the organelle metrics.

# `Inerst some lines of code explain the logic of this function; format like the above.`

In [10]:
def _get_region_metrics_3D(region_segs: List[np.ndarray], region_names: [str], intensity_img, mask: np.ndarray, scale: Union[tuple, None]=None) -> Tuple[Any, Any]:
    """
    Parameters
    ------------
    segmentation_img:
        a list of all 3d np.ndarray images of the segemented cell regions (e.g., whole cell, nucleus, cytoplasm, etc.)
    names:
        names or nicknames for the cell regions being analyzed
    intensity_img:
        a 3d np.ndarray image of the "raw" florescence intensity the segmentation was based on; for our use, this is the raw image with all the channels
        we will measure the intensity within the cell region being analyzed
    mask:
        a 3d np.ndarray image of the cell mask (or other mask of choice); used to create a "single cell" analysis

    Returns
    -------------
    pandas dataframe of containing regionprops measurements (columns) for each object in the segmentation image (rows) and the regionprops object

    """

    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]
    # add position
    properties = properties + ["centroid", "bbox"]
    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 
    # add shape measurements
    properties = properties + ["extent", "euler_number", "convex_area", "solidity", "axis_major_length"] # ,"feret_diameter_max", "axis_minor_length"]
    # add intensity values (used for quality checks)
    properties = properties + ["min_intensity", "max_intensity", "mean_intensity"]

    #######################
    ## ADD EXTRA PROPERTIES
    #######################
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])

    extra_properties = [standard_deviation_intensity]

    ##################
    ## RUN REGIONPROPS
    ##################
    intensity_input = np.moveaxis(intensity_img, 0, -1)

    props_stuff = []
    for idx, nm in enumerate(region_names):
        input_labels = apply_mask(region_segs[idx], mask)

        rp = regionprops(input_labels, 
                        intensity_image=intensity_input, 
                        extra_properties=extra_properties, 
                        spacing=scale)

        props = _my_props_to_dict(rp, 
                                label_image=input_labels, 
                                intensity_image=intensity_input, 
                                properties=properties, 
                                extra_properties=extra_properties,
                                spacing=scale)

        props_table = pd.DataFrame(props)

        surface_area_tab = pd.DataFrame(surface_area_from_props(input_labels, props))
        props_table.insert(11, "surface_area", surface_area_tab)
        props_table.insert(0, "region", nm)

        props_stuff.append(props_table)


    ################################################################
    ## ADD SKELETONIZATION OPTION FOR MEASURING LENGTH AND BRANCHING
    ################################################################
    #  # ETC.  skeletonize via cellprofiler /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/morphologicalskeleton.py
    #         if x.volumetric:
    #             y_data = skimage.morphology.skeletonize_3d(x_data)
    # /Users/ahenrie/Projects/Imaging/CellProfiler/cellprofiler/modules/measureobjectskeleton.py

    all_props_tab = pd.concat(props_stuff)
    all_props_tab.rename(columns={"area": "volume"}, inplace=True)
    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        all_props_tab.insert(loc=1, column="scale", value=f"{round_scale}")


    return all_props_tab

In [11]:
test_tab = _get_region_metrics_3D([cell_seg, cyto_seg, nuc_seg], ['cell', 'cyto', 'nuc'], raw_img_data, cell_seg, scale)

test_tab

,region,scale,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,...,mean_intensity-6,mean_intensity-7,standard_deviation_intensity-0,standard_deviation_intensity-1,standard_deviation_intensity-2,standard_deviation_intensity-3,standard_deviation_intensity-4,standard_deviation_intensity-5,standard_deviation_intensity-6,standard_deviation_intensity-7
0,cell,"(0.3891, 0.0799, 0.0799)",1,2.926375,29.817285,23.944586,0,0,0,16,...,615.328306,4042.493508,968.509916,1176.228311,3162.565822,3291.795243,1287.589652,2878.302425,1163.771714,2214.699198
0,cyto,"(0.3891, 0.0799, 0.0799)",1,3.040411,29.281374,22.647887,0,0,0,16,...,705.949372,4011.782276,419.711903,1296.688966,3551.936736,3718.003627,1396.749632,3100.415351,1298.872167,2363.657790
0,nuc,"(0.3891, 0.0799, 0.0799)",5,2.628936,31.602057,28.150081,0,284,242,16,...,357.386144,4216.287242,1048.054565,652.078905,453.954104,496.541180,887.180795,1842.257956,442.751818,1645.622597


#### Add function to `stats.py`

In [12]:
from infer_subc.utils.stats import get_region_metrics_3D

test_tab_final = get_region_metrics_3D([cell_seg, cyto_seg, nuc_seg], ['cell', 'cyto', 'nuc'], raw_img_data, cell_seg, scale)

test_tab_final

,region,scale,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,...,mean_intensity-6,mean_intensity-7,standard_deviation_intensity-0,standard_deviation_intensity-1,standard_deviation_intensity-2,standard_deviation_intensity-3,standard_deviation_intensity-4,standard_deviation_intensity-5,standard_deviation_intensity-6,standard_deviation_intensity-7
0,cell,"(0.3891, 0.0799, 0.0799)",1,2.926375,29.817285,23.944586,0,0,0,16,...,615.328306,4042.493508,968.509916,1176.228311,3162.565822,3291.795243,1287.589652,2878.302425,1163.771714,2214.699198
0,cyto,"(0.3891, 0.0799, 0.0799)",1,3.040411,29.281374,22.647887,0,0,0,16,...,705.949372,4011.782276,419.711903,1296.688966,3551.936736,3718.003627,1396.749632,3100.415351,1298.872167,2363.657790
0,nuc,"(0.3891, 0.0799, 0.0799)",5,2.628936,31.602057,28.150081,0,284,242,16,...,357.386144,4216.287242,1048.054565,652.078905,453.954104,496.541180,887.180795,1842.257956,442.751818,1645.622597


In [13]:
test_tab.equals(test_tab_final)

True

In [14]:
pd.set_option('display.max_columns', None)

test_tab_final

,region,scale,label,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,volume,surface_area,equivalent_diameter,extent,euler_number,convex_area,solidity,axis_major_length,min_intensity-0,min_intensity-1,min_intensity-2,min_intensity-3,min_intensity-4,min_intensity-5,min_intensity-6,min_intensity-7,max_intensity-0,max_intensity-1,max_intensity-2,max_intensity-3,max_intensity-4,max_intensity-5,max_intensity-6,max_intensity-7,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3,mean_intensity-4,mean_intensity-5,mean_intensity-6,mean_intensity-7,standard_deviation_intensity-0,standard_deviation_intensity-1,standard_deviation_intensity-2,standard_deviation_intensity-3,standard_deviation_intensity-4,standard_deviation_intensity-5,standard_deviation_intensity-6,standard_deviation_intensity-7
0,cell,"(0.3891, 0.0799, 0.0799)",1,2.926375,29.817285,23.944586,0,0,0,16,659,592,3835.846084,285939.000000,19.421712,0.247552,-13,6124.773560,0.626284,54.543657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131.0,8392.0,25817.0,43784.0,65535.0,25484.0,45278.0,43756.0,32471.0,779.060937,438.778382,1103.154897,968.052221,686.640820,2262.953316,615.328306,4042.493508,968.509916,1176.228311,3162.565822,3291.795243,1287.589652,2878.302425,1163.771714,2214.699198
0,cyto,"(0.3891, 0.0799, 0.0799)",1,3.040411,29.281374,22.647887,0,0,0,16,659,592,2946.076391,348005.531250,17.786144,0.190130,-13,6124.773560,0.481010,59.985268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,131.0,5862.0,25817.0,43784.0,65535.0,25484.0,45278.0,43756.0,32471.0,370.151939,465.353326,1394.939359,1197.639393,771.723197,2617.119110,705.949372,4011.782276,419.711903,1296.688966,3551.936736,3718.003627,1396.749632,3100.415351,1298.872167,2363.657790
0,nuc,"(0.3891, 0.0799, 0.0799)",5,2.628936,31.602057,28.150081,0,284,242,16,504,463,1037.508176,64248.140625,12.560231,0.537266,1,1135.780394,0.913476,21.135161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,446.0,8392.0,20433.0,15222.0,15249.0,15353.0,19653.0,7637.0,18477.0,2012.255031,367.603307,159.087391,236.961146,473.744757,1447.220691,357.386144,4216.287242,1048.054565,652.078905,453.954104,496.541180,887.180795,1842.257956,442.751818,1645.622597


Renaming the intensity numbers to match the organelles they came from